<a href="https://colab.research.google.com/github/pradeepragu/Gradtree-DP-SGD/blob/main/dpsgdFuntorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
1. select a batch of samples size L by selecting eachh sample to be in thhe lot independentity with probability L/n
2. for each samplee (x,y) in the batch, compute the gradient
3. Clip the the gradient to havee L2 norm at most C
4. Average the clipped gradients
5. Add Gaussian noise
6. Take a step in the negative direction of the resulting vector
"""


'\n1. select a batch of samples size L by selecting eachh sample to be in thhe lot independentity with probability L/n\n2. for each samplee (x,y) in the batch, compute the gradient\n3. Clip the the gradient to havee L2 norm at most C\n4. Average the clipped gradients \n5. Add Gaussian noise\n6. Take a step in the negative direction of the resulting vector\n'

In [1]:
pip install opacus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 kB 3.1 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim import Adam

# Preprocessing steps for image data (e.g., CIFAR-10)
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert the image to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image between [-1, 1]
])

# Download the CIFAR-10 dataset and split into training and test sets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# DataLoader for training and testing
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 170498071/170498071 [00:02<00:00, 57546698.80it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)  # CIFAR-10 images are 32x32; after pooling twice, size reduces to 8x8
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)  # 10 classes for CIFAR-10

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 + ReLU + Pooling
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + ReLU + Pooling
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))  # Fully connected layer
        x = F.relu(self.fc2(x))  # Fully connected layer
        x = self.fc3(x)  # Output layer
        return x


In [4]:
#hyperparameters

# Loss function
def loss_fn(predictions, targets):
    return F.cross_entropy(predictions, targets)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model, optimizer, and other hyperparameters
model = CNN().to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
# DP-SGD parameters

max_grad_norm = 1.0
target_epsilon = 1.0  # Privacy budget target (fixed at 1,3,6)
delta = 1e-5          # Delta value
sample_rate = batch_size / len(train_loader.dataset)
num_epochs = 50
MAX_PHYSICAL_BATCH_SIZE = 256

In [5]:
# Import the GaussianAccountant class from your accountant file
import warnings
from opacus.accountants import IAccountant
from opacus.accountants.analysis import gdp as privacy_analysis
from opacus.accountants import create_accountant
from typing import Optional
class GaussianAccountant(IAccountant):
    def __init__(self):
        warnings.warn(
            "GDP accounting is experimental and can underestimate privacy expenditure."
            "Proceed with caution. More details: https://arxiv.org/pdf/2106.02848.pdf"
        )
        super().__init__()

    def step(self, *, noise_multiplier: float, sample_rate: float):
        if len(self.history) >= 1:
            last_noise_multiplier, last_sample_rate, num_steps = self.history.pop()
            if (
                last_noise_multiplier != noise_multiplier
                or last_sample_rate != sample_rate
            ):
                raise ValueError(
                    "Noise multiplier and sample rate have to stay constant in GaussianAccountant."
                )
            else:
                self.history = [
                    (last_noise_multiplier, last_sample_rate, num_steps + 1)
                ]

        else:
            self.history = [(noise_multiplier, sample_rate, 1)]

    def get_epsilon(self, delta: float, poisson: bool = True) -> float:
        """
        Return privacy budget (epsilon) expended so far.

        Args:
            delta: target delta
            poisson: ``True`` is input batches was sampled via Poisson sampling,
                ``False`` otherwise
        """

        compute_eps = (
            privacy_analysis.compute_eps_poisson
            if poisson
            else privacy_analysis.compute_eps_uniform
        )
        noise_multiplier, sample_rate, num_steps = self.history[-1]
        return compute_eps(
            steps=num_steps,
            noise_multiplier=noise_multiplier,
            sample_rate=sample_rate,
            delta=delta,
        )

    def __len__(self):
        return len(self.history)

    @classmethod
    def mechanism(cls) -> str:
        return "gdp"


# Initialize the GaussianAccountant
accountant = GaussianAccountant()

MAX_SIGMA = 1e6

# Get noise multiplier function
def get_noise_multiplier(
    *,
    target_epsilon: float,
    target_delta: float,
    sample_rate: float,
    epochs: Optional[int] = None,
    steps: Optional[int] = None,
    accountant: str = "gdp",
    epsilon_tolerance: float = 0.01,
    **kwargs,
) -> float:
    if (steps is None) == (epochs is None):
        raise ValueError("get_noise_multiplier takes as input EITHER a number of steps or a number of epochs")
    if steps is None:
        steps = int(epochs / sample_rate)

    eps_high = float("inf")
    accountant = create_accountant(mechanism=accountant)

    sigma_low, sigma_high = 0, 10
    while eps_high > target_epsilon:
        sigma_high = 2 * sigma_high
        accountant.history = [(sigma_high, sample_rate, steps)]
        eps_high = accountant.get_epsilon(delta=target_delta, **kwargs)
        if sigma_high > MAX_SIGMA:
            raise ValueError("The privacy budget is too low.")

    while target_epsilon - eps_high > epsilon_tolerance:
        sigma = (sigma_low + sigma_high) / 2
        accountant.history = [(sigma, sample_rate, steps)]
        eps = accountant.get_epsilon(delta=target_delta, **kwargs)

        if eps < target_epsilon:
            sigma_high = sigma
            eps_high = eps
        else:
            sigma_low = sigma

    return sigma_high
# Calculate the noise multiplier before training using the total number of steps
noise_multiplier = get_noise_multiplier(
    target_epsilon=target_epsilon,
    target_delta=delta,
    sample_rate=sample_rate,
    epochs=num_epochs,
)
print(f"Calculated noise multiplier for fixed privacy budget: {noise_multiplier}")

Calculated noise multiplier for fixed privacy budget: 1.1572265625


<ipython-input-5-9d430db62b61>:9: UserWarning: GDP accounting is experimental and can underestimate privacy expenditure.Proceed with caution. More details: https://arxiv.org/pdf/2106.02848.pdf
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/gdp.py:23: UserWarning: GDP accounting is experimental and can underestimate privacy expenditure.Proceed with caution. More details: https://arxiv.org/pdf/2106.02848.pdf
  warnings.warn(


In [6]:
from functorch import make_functional_with_buffers, vmap, grad
#This will separate state (the parameters) from the model and turn the model into a pure function
fmodel, params, buffers = make_functional_with_buffers(model)
#the model has become the stateless FunctionalModuleWithBuffers
fmodel
#the model parameters now exist independently of the model, stored as a tuple
for x in params:
  print(f"{x.shape}")

print(f"\n{type(params)}")
#function to compute the loss of the model given a single input rather than a batch of inputs
def compute_loss_stateless_model (params, buffers, sample, target):
    batch = sample.unsqueeze(0)
    targets = target.unsqueeze(0)

    predictions = fmodel(params, buffers, batch)
    loss = loss_fn(predictions, targets)
    return loss
#let’s use functorch’s grad to create a new function that computes the gradient with respect to the first argument of compute_loss (i.e. the params).
ft_compute_grad = grad(compute_loss_stateless_model)
#ft_compute_grad was used to find the gradient for single sample. now using vmap we extend that to entire batch
ft_compute_sample_grad = vmap(ft_compute_grad, in_dims=(None, None, 0, 0))



torch.Size([32, 3, 3, 3])
torch.Size([32])
torch.Size([64, 32, 3, 3])
torch.Size([64])
torch.Size([256, 4096])
torch.Size([256])
torch.Size([128, 256])
torch.Size([128])
torch.Size([10, 128])
torch.Size([10])

<class 'tuple'>


<ipython-input-6-b53a6d5af22e>:3: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.make_functional_with_buffers` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.functional_call` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  fmodel, params, buffers = make_functional_with_buffers(model)
<ipython-input-6-b53a6d5af22e>:20: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.grad` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.func.grad` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  ft_compute_grad = grad(compute_loss_stateless_model)
<ipython-input-

In [7]:
# Modify the clip_gradients function to handle single examples
def clip_gradients_and_add_noise(params, max_grad_norm, per_sample_grads):
    """Clip per-sample gradients to a maximum L2 norm."""
    # Compute the L2 norm of each per-sample gradient
    total_grad_norms = torch.stack([
        torch.norm(torch.stack([torch.norm(g) for g in sample_grads]))
        for sample_grads in zip(*per_sample_grads)
    ])

    # Compute the clipping factor for each sample
    clip_coef = max_grad_norm / (total_grad_norms + 1e-6)
    clip_coef_clamped = torch.clamp(clip_coef, max=1.0)

    # Clip the gradients
    clipped_grads = [
        [g * c for g in sample_grads]
        for sample_grads, c in zip(zip(*per_sample_grads), clip_coef_clamped)
    ]

    # Corrected gradient aggregation step
    aggregated_grads = [torch.sum(torch.stack(grad), dim=0) for grad in zip(*clipped_grads)]

    # Add noise to the aggregated gradients
    noised_grads = [
        g + noise_multiplier * max_grad_norm * torch.randn_like(g) / batch_size
        for g in aggregated_grads
    ]

    return noised_grads


In [8]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [11]:
# Update the train_step function

def train_step(model, optimizer, data, targets):
    per_sample_grads = ft_compute_sample_grad(params, buffers, data, targets)
    noised_grads = clip_gradients_and_add_noise(params, max_grad_norm, per_sample_grads)

    optimizer.zero_grad()
    for param, noised_grad in zip(model.parameters(), noised_grads):
        param.grad = noised_grad
    optimizer.step()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    return test_loss, accuracy


In [12]:
# Training loop
steps = 0
losses = []
epsilons = []
train_accuracies = []

for epoch in range(num_epochs):  # Train for full epochs
    model.train()
    epoch_loss = 0
    correct_train = 0
    total_train = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Perform a training step
        train_step(model, optimizer, data, target)
        steps += 1

        # Update accountant with current noise multiplier and sample rate
        accountant.step(noise_multiplier=noise_multiplier, sample_rate=sample_rate)

        # Compute loss for the current batch
        outputs = model(data)
        loss = loss_fn(outputs, target)
        epoch_loss += loss.item()

        # Compute the number of correct predictions in the training batch
        pred = outputs.argmax(dim=1, keepdim=True)
        correct_train += pred.eq(target.view_as(pred)).sum().item()
        total_train += target.size(0)

    # Compute average loss over the training data
    losses.append(epoch_loss / len(train_loader))

    # Calculate training accuracy
    train_accuracy = 100. * correct_train / total_train
    train_accuracies.append(train_accuracy)

    # Evaluate on test set
    test_loss, test_accuracy = test(model, device, test_loader)

    # Get the current epsilon from GaussianAccountant
    epsilon = accountant.get_epsilon(delta=delta, poisson=True)  # Use Poisson sampling
    epsilons.append(epsilon)

    # Print results for the current epoch
    print(f'Epoch {epoch+1}: Train Loss: {epoch_loss / len(train_loader):.4f}, '
          f'Train Accuracy: {train_accuracy:.2f}%, '
          f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%, '
          f'Epsilon: {epsilon:.4f}')

Epoch 1: Train Loss: 64.1244, Train Accuracy: 10.67%, Test Loss: 303.5601, Test Accuracy: 10.00%, Epsilon: 0.1176


KeyboardInterrupt: 